In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from sklearn.mixture import GaussianMixture 
#import testing
import sys
sys.path.append("../")
import selection_tests

import itertools

In [2]:
data = pd.read_stata("all_plans_c_bonus.dta")
data['log_enroll'] = np.log(data['enr_c'])
data['enr_total'] = data['enr_c'] + data['enr_FFS']
data = data[data['year'] > 2011]
print(data.columns)

print(data[data['double_bonus']==1]['bmFFS'].mean())
print(data['bmFFS'].mean())
print(data['star_C2'].mean())

Index(['index', 'ssa', 'state', 'county', 'year', 'enr_FFS', 'enr_c',
       'hhi_ins', 'hhi_ins_noSNP', 'ins_parent', 'ins_parent_noSNPs',
       'ins_plans', 'HMO_share', 'PPO_share', 'qual_2012', 'qual_2013',
       'qual_2014', 'qual_2015', 'partaenrollment', 'partb_enrollment',
       'prescription_drugs', 'prev_comp_dental', 'eye_exams', 'hearing_exams',
       'deductible', 'partb_premium', 'plan_premium', 'partd_premium', 'OOPC',
       'risk_pub_p', 'bid_pub_p', 'rebate_pub_p', 'risk_pub_c', 'bid_pub_c',
       'rebate_pub_c', 'star_C2', 'star_CD2', 'bmFFS', 'bm_ns', 'risk_FFS',
       'FFS_AB', 'FFS_AB_rs', 'buydown', 'OOPC_noprem', 'extras', 'quartile',
       'bid_pub_p_nominal', 'bid_pub_c_nominal', 'rebate_pub_p_nominal',
       'rebate_pub_c_nominal', 'bmFFS_nominal', 'FFS_AB_nominal',
       'bm_ns_nominal', 'OOPC_nominal', 'plan_premium_nominal',
       'partd_premium_nominal', 'buydown_nominal', 'rebate_std',
       'benchmark_diff', 'benchmark_diff_n', 'benchmark_dif

In [3]:
def join_print(t1,t2,
               summary_xs=['double_bonus', 'bmFFS',  'ins_parent_noSNPs',  'FFS_AB', 
                           'unemploy_rt', 'pc_income', 'log_risk_FFS', 'log_risk_pub_c', 
                           'risk_pub_c', 'rebate_pub_c'] ):
    table =  pd.DataFrame(index=summary_xs)
    table['1'] = t1
    table['2'] = t2
    
    for row in table.itertuples():
        listrow = list(row)
        print('\\textbf{%s}'%listrow[0].replace('_','\\_'),end='')
        for i in range(len(listrow)-1):
            print(' & %.4f '%listrow[i+1],end='')
        print('\\\\')

summary_xs1=['double_bonus', 'bmFFS', 'bm_ns','star_C2', 'ins_parent_noSNPs',  'FFS_AB', 
                           'unemploy_rt', 'pc_income', 'log_risk_FFS', 'log_risk_pub_c', 
                           'risk_pub_c', 'rebate_pub_c']

join_print(data[summary_xs1].mean(),data[summary_xs1].std())

\textbf{double\_bonus} & 0.0687  & 0.2530 \\
\textbf{bmFFS} & 806.3442  & 59.9781 \\
\textbf{ins\_parent\_noSNPs} & 2.6149  & 1.5398 \\
\textbf{FFS\_AB} & 703.6609  & 95.1605 \\
\textbf{unemploy\_rt} & 6.2953  & 2.4442 \\
\textbf{pc\_income} & 39.5061  & 10.7500 \\
\textbf{log\_risk\_FFS} & -3.8737  & 7.5688 \\
\textbf{log\_risk\_pub\_c} & -6.9952  & 12.9910 \\
\textbf{risk\_pub\_c} & 0.9403  & 0.1217 \\
\textbf{rebate\_pub\_c} & 41.3758  & 32.7914 \\


In [4]:
#https://www.statsmodels.org/dev/examples/notebooks/generated/glm_weights.html

def drop_data(data,y_name,x_name,absorb=None):
    data = data.copy()
    data = data[y_name + x_name + absorb]
    missing_vals = ~data.isnull().max(axis=1)
    data = data[missing_vals]
    data = data[data['year'].groupby(data['ssa']).transform('count')>=11]
    return data
    

def demean(y_name,x_name,data=None,absorb=None,cluster=None): 

    y,X = data[ y_name], data[ x_name ]
    
    y_dot = y.copy()
    X_dot = X.copy()
    
    ybar = y.mean()
    Xbar = X.mean()

    
    for effect in absorb:
        y_dot = y_dot - y.groupby(data[effect]).transform('mean')
        X_dot = X_dot - X.groupby(data[effect]).transform('mean') 
    
    y_dot = y_dot + ybar
    X_dot = X_dot + Xbar
    return y_dot, X_dot

In [5]:
model1_x = ['double_bonus','log_risk_pub_c', 'bm_ns','star_C2',
            'FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income']

model2_x = ['bmFFS','log_risk_pub_c', 'bm_ns','star_C2',
            'FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income']
    
model3_x = ['bmFFS', 'log_risk_pub_c', 'bm_ns','star_C2',
            'FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income','bmFFS*log_risk_pub_c']


model4_x = ['bmFFS',  'log_risk_pub_c', 'bm_ns','star_C2',
            'FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income','bmFFS*c']

model_xs = [model1_x,model2_x,model3_x,model4_x]



def create_latent(y_dot, X_dot, missing_vals):
    gmmodel = GaussianMixture (n_components=2)
    
    #setup data
    data_stack = X_dot[['log_risk_pub_c']].copy()
    data_stack['y'] = y_dot
    
    #predict
    classify = GaussianMixture (n_components=2).fit(data_stack)
    c = np.array(classify.predict(data_stack))
    return c
    
    
def latent_stats(c,data,x_name,missing_vals):
    summary_xs=['double_bonus', 'bmFFS',  'ins_parent_noSNPs',  'FFS_AB', 'bm_ns','star_C2', 
                           'unemploy_rt', 'pc_income', 'log_risk_FFS', 'log_risk_pub_c', 
                           'risk_pub_c', 'rebate_pub_c'] 
    print(c.sum()/c.shape[0] )
    join_print(data[missing_vals][summary_xs][c==0].mean() ,data[missing_vals][summary_xs][c==1].mean()  )


def setup_data(y_name,model_xs,data):
     #get the super set of all the model names
    all_xs = set()
    for model_x in model_xs:
        all_xs = all_xs.union(set(model_x))
    all_xs = list(all_xs)
    
    #subtract out the columns that are not in the data
    x_name = []
    for col in data.columns:
        if col in all_xs:
            x_name.append(col)
    print(x_name)
    
    #clean the data
    y_dot, X_dot = demean(y_name,x_name, data=data,absorb=['ssa','year'])
    missing_vals = ~data[y_name + x_name].isnull().max(axis=1)
    y_dot, X_dot = y_dot[missing_vals],X_dot[missing_vals]
    return y_dot,X_dot,x_name,missing_vals



def return_results(y_name,model_xs,data,weights=True):
    y_dot,X_dot,x_name,missing_vals = setup_data(y_name,model_xs,data)
    
    #setup the latent variable
    c =  create_latent(y_dot, X_dot, missing_vals)
    latent_stats(c,data,x_name,missing_vals)
    
    #setup interactions
    X_dot['bmFFS*c'] = X_dot['bmFFS']*c
    X_dot['bmFFS*log_risk_pub_c'] = X_dot['bmFFS']*X_dot['log_risk_pub_c']
    
    print('-----')
    params = []
    se = []
    for model_x in model_xs:
        if weights:
            var_weights = np.array( data['enr_c'][missing_vals] )
            X_dot_m = X_dot[model_x].copy()
            model = sm.GLM(y_dot,X_dot_m,var_weights=var_weights)
            model_fit = model.fit()
            params.append(model_fit.params)
            se.append(model_fit.bse)
    table  = pd.DataFrame(index=x_name + ['bmFFS*c','bmFFS*log_risk_pub_c'])
    col_names = []
    for i in range(len(model_xs)):
        table['params %i'%(i+1)]  = params[i]
        table['se %i'%(i+1)]  = se[i]
    return table
    

table = return_results(['log_enroll'],model_xs,data)

['ins_parent_noSNPs', 'star_C2', 'bmFFS', 'bm_ns', 'FFS_AB', 'unemploy_rt', 'pc_income', 'log_risk_FFS', 'log_risk_pub_c', 'double_bonus']
0.744511968250031
\textbf{double\_bonus} & 0.0036  & 0.0910 \\
\textbf{bmFFS} & 804.7476  & 806.8893 \\
\textbf{ins\_parent\_noSNPs} & 1.7920  & 2.8973 \\
\textbf{FFS\_AB} & 721.2720  & 697.6155 \\
\textbf{unemploy\_rt} & 6.5507  & 6.2077 \\
\textbf{pc\_income} & 38.1133  & 39.9841 \\
\textbf{log\_risk\_FFS} & -4.4944  & -3.6608 \\
\textbf{log\_risk\_pub\_c} & -8.5549  & -6.4600 \\
\textbf{risk\_pub\_c} & 0.9302  & 0.9437 \\
\textbf{rebate\_pub\_c} & 30.2175  & 45.2049 \\
-----


In [6]:
def table_to_latex(table):
    num_col = len(table.columns)
    print('\\begin{tabular}{l',end='')
    for i in range(int(len(table.columns)/2-1)):
        print('cc|',end='')
    print('cc}')
    print('\\toprule')
    print('{}',end='')
    model = 1
    while model <= len(table.columns)/2:
        print('& \\textbf{coef %s} & \\textbf{se %s}'%(model,model),end='' )
        model = model +1 
    print('\\\\')
    print('\\midrule')

    
    for row in table.itertuples():
        listrow = list(row)
        print('\\textbf{%s}'%listrow[0].replace('_','\\_'),end='')
        for i in range(len(listrow)-1):
            print(' & %.4f '%listrow[i+1],end='')
        print('\\\\')
    print('\\bottomrule')
    print('\\end{tabular}')
    
table_to_latex(table)

\begin{tabular}{lcc|cc|cc|cc}
\toprule
{}& \textbf{coef 1} & \textbf{se 1}& \textbf{coef 2} & \textbf{se 2}& \textbf{coef 3} & \textbf{se 3}& \textbf{coef 4} & \textbf{se 4}\\
\midrule
\textbf{ins\_parent\_noSNPs} & 0.0166  & 0.0013  & 0.0159  & 0.0013  & 0.0161  & 0.0013  & 0.0161  & 0.0013 \\
\textbf{star\_C2} & -0.0014  & 0.0002  & -0.0014  & 0.0002  & -0.0014  & 0.0002  & -0.0014  & 0.0002 \\
\textbf{bmFFS} & nan  & nan  & 0.0002  & 0.0001  & 0.0001  & 0.0001  & 0.0009  & 0.0002 \\
\textbf{bm\_ns} & 0.0006  & 0.0000  & 0.0005  & 0.0001  & 0.0005  & 0.0001  & 0.0005  & 0.0001 \\
\textbf{FFS\_AB} & -0.0001  & 0.0000  & -0.0001  & 0.0000  & -0.0001  & 0.0000  & -0.0001  & 0.0000 \\
\textbf{unemploy\_rt} & -0.0112  & 0.0015  & -0.0116  & 0.0016  & -0.0119  & 0.0016  & -0.0116  & 0.0016 \\
\textbf{pc\_income} & -0.0045  & 0.0005  & -0.0051  & 0.0005  & -0.0052  & 0.0005  & -0.0052  & 0.0005 \\
\textbf{log\_risk\_FFS} & 0.0027  & 0.0007  & 0.0030  & 0.0007  & 0.0032  & 0.0007  & 0.0030  

In [7]:
class GLS_LL(GenericLikelihoodModel):

    def __init__(self, *args, model=None, **kwargs):
        super(GLS_LL, self).__init__(*args, **kwargs)
        self.model = model
        
    def loglikeobs(self, params, scale=None):
        """
        Evaluate the log-likelihood for a generalized linear model.
        """
        model = self.model
        scale = sm.tsa.stattools.float_like(scale, "scale", optional=True)
        lin_pred = np.dot(model.exog, params) + model._offset_exposure
        expval = model.family.link.inverse(lin_pred)
        if scale is None:
            scale = model.estimate_scale(expval)
        llf = model.family.loglike_obs(model.endog, expval, model.var_weights,
                                  scale)
        return llf



#this is janky as f.... need to fix it...
#this is janky as f.... need to fix it...
def setup_test(y_dot,X_dot,
    model1_cov = [],
    model2_cov = []):
    
    #model 1
    #weights = np.array( data['enr_c_mean'][missing_vals] )
    m1 = sm.GLM(y_dot,X_dot[model1_cov])#,var_weights=weights)
    m1_fit = m1.fit()

    #model2
    m2 = sm.GLM(y_dot,X_dot[model2_cov])#,var_weights=weights)
    m2_fit = m2.fit()

    model1 = GLS_LL(y_dot, X_dot[model1_cov], model=m1)
    ll1 = model1.loglikeobs(m1_fit.params)
    grad1 = model1.score_obs(m1_fit.params)
    hess1 = model1.hessian(m1_fit.params)
    params1 = m1_fit.params

    model2 = GLS_LL(y_dot, X_dot[model2_cov], model=m2)

    ll2 = model2.loglikeobs(m2_fit.params)
    grad2 = model2.score_obs(m2_fit.params)
    hess2 = model2.hessian(m2_fit.params)
    params2 = m2_fit.params
    
    return  ll1, grad1, hess1, params1, ll2, grad2, hess2, params2



def pairwise_tests(y_name,model_xs,data):
    y_dot,X_dot,x_name,missing_vals = setup_data(y_name,model_xs,data)
    
    #setup the latent variable
    c =  create_latent(y_dot, X_dot, missing_vals)
    
    #setup interactions
    X_dot['bmFFS*c'] = X_dot['bmFFS']*c
    X_dot['bmFFS*log_risk_pub_c'] = X_dot['bmFFS']*X_dot['log_risk_pub_c']
    
    #TODO fix this so that it does all the comparison
    combos = list(itertools.combinations(model_xs,2))
    labels = [ 'll'+ str(i+1) for i in range(len(model_xs))]
    label_combos = list(itertools.combinations(labels,2))
    res = []
    for i in range(len(combos)):
        combo = combos[i]
        label_combo = label_combos[i]
        model1_x = combo[0]
        model2_x = combo[1]
        setup_test_i = lambda yn,xn : setup_test(yn,xn,model1_cov = model1_x, model2_cov= model2_x)
        test_stat,res1,res2,res3 = selection_tests.test_results(y_dot,X_dot,setup_test_i)
        res.append( [label_combo,test_stat,res1,res2,res3])
        print(label_combo,test_stat,res1,res2,res3)
    #print_pairwise_tests(res)
    return res

    
res = pairwise_tests(['log_enroll'],model_xs,data)

['ins_parent_noSNPs', 'star_C2', 'bmFFS', 'bm_ns', 'FFS_AB', 'unemploy_rt', 'pc_income', 'log_risk_FFS', 'log_risk_pub_c', 'double_bonus']
('ll1', 'll2') -2.0965416567514286 95 90 90
('ll1', 'll3') -2.111732553866967 95 85 90
('ll1', 'll4') -3.4418968932339156 99 90 99
('ll2', 'll3') -0.09146312303085556 85 90 99
('ll2', 'll4') -2.142668790625963 95 85 90
('ll3', 'll4') -2.1537105767995697 95 90 95


In [8]:
def print_pairwise_tests(res):
    
    print('\\begin{center}')
    print('\\begin{tabular}{ccc|c|ccc}')
    print('\\toprule')
    print('{} & \\textbf{Hypotheses} & {} & {}  & {} & \\textbf{p Value}  & {}  \\\\')
    print('\\textbf{H0} & \\textbf{H1} & \\textbf{H2} & \\textbf{Selection} & \\textbf{Shi (2015)} & \\textbf{Classical} & \\textbf{Bootstrap}\\\\')
    print('\\midrule')
    for row in res:
        label_combo,test_stat,classical,shi,boot = row
        print( '$%s=%s$ & $%s>%s$ & $%s<%s$ &'%(label_combo+label_combo+label_combo),end='' )
        if test_stat > 0:
            print(' $H1$ ', end='')
        if test_stat <= 0:
            print(' $H2$ ',end='')
        for sig in [shi,classical,boot]:
                
            if sig == 85:
                print(' & $-$',end='')
            if sig == 90:
                print( ' & $[90,95)$',end='')
            if sig == 95:
                print( ' & $[95,100)$',end='')
            if sig == 99:
                print(' & $[99,100)$ ',end='')
        print('\\\\')
    print('\\bottomrule')
    print('\\end{tabular}')
    print('\\end{center}')

print_pairwise_tests(res)

\begin{center}
\begin{tabular}{ccc|c|ccc}
\toprule
{} & \textbf{Hypotheses} & {} & {}  & {} & \textbf{p Value}  & {}  \\
\textbf{H0} & \textbf{H1} & \textbf{H2} & \textbf{Selection} & \textbf{Shi (2015)} & \textbf{Classical} & \textbf{Bootstrap}\\
\midrule
$ll1=ll2$ & $ll1>ll2$ & $ll1<ll2$ & $H2$  & $[90,95)$ & $[95,100)$ & $[90,95)$\\
$ll1=ll3$ & $ll1>ll3$ & $ll1<ll3$ & $H2$  & $-$ & $[95,100)$ & $[90,95)$\\
$ll1=ll4$ & $ll1>ll4$ & $ll1<ll4$ & $H2$  & $[90,95)$ & $[99,100)$  & $[99,100)$ \\
$ll2=ll3$ & $ll2>ll3$ & $ll2<ll3$ & $H2$  & $[90,95)$ & $-$ & $[99,100)$ \\
$ll2=ll4$ & $ll2>ll4$ & $ll2<ll4$ & $H2$  & $-$ & $[95,100)$ & $[90,95)$\\
$ll3=ll4$ & $ll3>ll4$ & $ll3<ll4$ & $H2$  & $[90,95)$ & $[95,100)$ & $[95,100)$\\
\bottomrule
\end{tabular}
\end{center}
